<a href="https://colab.research.google.com/github/Andradeg271/predict-sleep-quality/blob/main/Sleep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

EXECUTE THIS CELL TO CREATE AND TRAIN THE MODEL 🤖

In [ ]:
# Import Statements

from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder,QuantileTransformer
from sklearn.model_selection import GridSearchCV
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from ipywidgets import interact, Dropdown
import ipywidgets as widgets



# Load the dataset and remove the ID column

df = pd.read_csv('sleep_dataset.csv')
df.drop('Person ID',axis=1,inplace=True)


# Define numerical and categorical columns

numeric_col = ['Age','Sleep Duration','Physical Activity Level','Stress Level','Heart Rate','Daily Steps']
categorical_col = ['Gender','Occupation','Sleep Disorder','BMI Category']


# Set up a column transformer to process the numerical and categorical columns

pre_processor = ColumnTransformer(
    transformers=[
        ('num', QuantileTransformer(n_quantiles=249), numeric_col),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_col)
    ])


# Create a Pipeline

pipe = Pipeline([
    ('preprocessor', pre_processor),
    ('classifier', RandomForestRegressor())
])


# Create a grid

grid = GridSearchCV(
    estimator=pipe,
    param_grid={
        'classifier__n_estimators': [100, 200, 300],
        'classifier__max_depth': [None, 10, 20],
        },
    cv = 3)


# Feed the model

X = df.drop('Quality of Sleep',axis=1)
y = df['Quality of Sleep']

grid.fit(X,y)

EXECUTE THIS CELL TO SEE YOUR SLEEP QUALITY ⏰

In [ ]:
# Function to collect user information

def get_user_input():

    age = float(input("Age: "))
    sleep_duration = float(input("Sleep Duration (hours): "))
    physical_activity_level = float(input("Physical Activity Level (1 to 100): "))
    stress_level = float(input("Stress Level (1 to 10): "))
    heart_rate = float(input("Heart Rate (beats per minute): "))
    daily_steps = float(input("Daily Steps: "))
    gender = input("Gender: ")
    occupation = input("Occupation: ")
    sleep_disorder = input("Sleep Disorder (eg. Insomnia, Sleep Apnea, None, etc.): ")
    bmi_category = input("BMI Category (Underweight/Normal/Overweight/Obese): ")
    date = input("Date (DD/MM): ")


    # Creating a dataset with the user's information

    user_data = {
        'Age': age,
        'Sleep Duration': sleep_duration,
        'Physical Activity Level': physical_activity_level,
        'Stress Level': stress_level,
        'Heart Rate': heart_rate,
        'Daily Steps': daily_steps,
        'Gender': gender,
        'Occupation': occupation,
        'Sleep Disorder': sleep_disorder,
        'BMI Category': bmi_category,
        'Date': date
        }
    return user_data


# Storing the dataset to a variable

user_information = pd.DataFrame([get_user_input()])


# Predict the quality of sleep and combine it to the users dataset

quality_sleep = grid.predict(user_information.drop('Date',axis = 1))
user_dataset = pd.concat([user_information, pd.DataFrame({'Quality of Sleep': quality_sleep})], axis=1)


# Printing the quality of sleep

print(f'Your quality of sleep was: {quality_sleep}')



EXECUTE THIS CELL TO SAVE YOUR RESULTS TO A CSV FILE 📁

In [ ]:
# Function to append the users data to an existing CSV file if it exists

def save_user_data(row):
  try:
    df_user = pd.read_csv('sleep_dataset_user.csv')
    df_user = pd.concat([df_user, user_dataset], ignore_index=True)
    df_user.to_csv('sleep_dataset_user.csv', index=False)
    print('Your data was successfully appended to the existing file.')

  except FileNotFoundError:
    print('There was no existent user file found. Make sure you have uploaded a csv file named "sleep_dataset_user.csv" and try again.')
    return

# Ask user whether to create a new file or append to existing one

save_file = input('Do you want to save your data in a new file, or upload an existing one to add a new row?\nType "n" to create a new file or "u" to update an existing file.\n')


# Save file

if save_file == 'n':
  df_user = user_dataset.to_csv('sleep_dataset_user.csv',index=False)
  print('Your dataset was saved under the file name "sleep_dataset_user.csv"')


# Update and combine file

elif save_file == 'u':
  save_user_data(user_dataset)


EXECUTE THIS CELL TO SEE YOUR SLEEP OVER TIME ✅

In [ ]:
# Load user data
df_user = pd.read_csv('sleep_dataset_user.csv')

# Define available options for visualization
columns_to_plot = {
    'Quality of Sleep': 'Quality of Sleep',
    'Sleep Duration (hours)': 'Sleep Duration',
    'Physical Activity Level': 'Physical Activity Level',
    'Stress Level': 'Stress Level',
    'Heart Rate': 'Heart Rate',
    'Daily Steps': 'Daily Steps'
}

# Function to create the plot based on selection
def plot_sleep_data(selected_metric):
    x = df_user['Date']
    y = df_user[columns_to_plot[selected_metric]]

    plt.figure(figsize=(10, 6))
    plt.plot(x, y, marker='o', linestyle='-', color='teal')
    plt.title(f'{selected_metric} over Time')
    plt.xlabel('Date')
    plt.ylabel(selected_metric)
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Create the dropdown widget
interact(plot_sleep_data, selected_metric=widgets.Dropdown(
    options=list(columns_to_plot.keys()),
    description='Select metric:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
))